7/2/18:
this is ment to be a temporary notebook where I will experiment with how to run the mimic SOFA score sql code. The code requires having a number of materialized views open, and this is somthing i am unfamiliar with how to handle in jupyter. The purpose of the notebook will be to learn how to utilize these materialized views to run the sofa score script.

upon completion of this, i intend on deleting this file and incorporating the lessons I learn into my labnotebook and my main 03-clinical_variables_streamlined.ipnb notebook

In [1]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [44]:
import psycopg2
conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

time: 2.2 s


In [20]:
query_schema

'SET search_path to mimiciii;'

time: 1.91 ms


## cohort

In [3]:
#patients of interest from rotation_cohort_generation

final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/06072018_final_pt_df2.csv', index_col=0)

patients= list(final_pt_df2['subject_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 57.7 ms


## functions

In [4]:
def sql_exe_show(sql_sentence):
    cur.execute(sql_sentence)
    rows = cur.fetchall()
    col = []
    for i in range(len(cur.description)):
        col.append(cur.description[i][0])
    table = pd.DataFrame(rows,columns=col)
    return table

#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['icustay_id'].isin(icustay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df

time: 13.8 ms


from the github:
    
How to use code in this folder
This folder generates materialized views for a variety of severity of illness scores (detailed below). The queries make use of materialized views which aggregate data from the first day of a patient's ICU stay. To run the code, it is necessary to:

Run the scripts which generate the constituent materialized views
echodata - generated by /concepts/echo-data.sql
ventdurations - generated by /concepts/durations/ventilation-durations.sql - (needed for subsequent view)
vitalsfirstday - generated by /concepts/firstday/vitals-first-day.sql
uofirstday - generated by /concepts/firstday/urine-output-first-day.sql
ventfirstday - generated by /concepts/firstday/ventilated-first-day.sql
gcsfirstday - generated by /concepts/firstday/gcs-first-day.sql
labsfirstday - generated by /concepts/firstday/labs-first-day.sql
bloodgasfirstday - generated by /concepts/firstday/blood-gas-first-day.sql - (needed for subsequent view)
bloodgasfirstdayarterial - generated by /concepts/firstday/blood-gas-first-day-arterial.sql


for sofa specifically:

--  1) uofirstday - generated by urine-output-first-day.sql
--  2) vitalsfirstday - generated by vitals-first-day.sql
--  3) gcsfirstday - generated by gcs-first-day.sql
--  4) labsfirstday - generated by labs-first-day.sql
--  5) bloodgasfirstdayarterial - generated by blood-gas-first-day-arterial.sql
--  6) echodata - generated by echo-data.sql
--  7) ventdurations - generated by ventilation-durations.sql


Run the script for the severity of illness score you are interested in
OASIS - oasis.sql
SAPS - saps.sql
SOFA - sofa.sql

file location of mimic first day codes (just add on the ending posted above next to each script):
/Users/geickelb1/Documents/GitHub/mimic-code


NOTE: in order to run off dr. Luo's server, i need to add in mimiciii. before each table reference in the script. This appears to be because I don't have write access to the database. 

NOTE2: i will be adjusting the files within the mimic-code folder, but I copied the folder prior to my desktop incase i need to revert these.


In [ ]:
#example of how i've been running my materialized view scripts. 

f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/ventparam_all48.sql', 'r') 
ventparamdata = f.read()
ventparam_all48_sql = ventparamdata#.format(tuple(icustay_id))  
cur.execute('rollback')
cur.execute(ventparam_all48_sql)  #I THINK that if i just execute all the scripts that should be sufficient.
#ventparamTable= sql_exe_show('select * from ventparam_all48;')
#ventparam_all48_df= sql_to_df_icu(ventparamTable)

### executing materialized views:

In [23]:
#urine output

f = open('/Users/geickelb1/Documents/GitHub/mimic-code/concepts/firstday/urine-output-first-day.sql', 'r') 
urine_output_first_day = f.read()
cur.execute('rollback')
cur.execute(urine_output_first_day)

time: 4.27 s


In [24]:
#vitals
f = open('/Users/geickelb1/Documents/GitHub/mimic-code/concepts/firstday/vitals-first-day.sql', 'r') 
vitals_first_day = f.read()
cur.execute('rollback')
cur.execute(vitals_first_day)

time: 8min 36s


In [28]:
#gcs
f = open('/Users/geickelb1/Documents/GitHub/mimic-code/concepts/firstday/gcs-first-day.sql', 'r') 
gcs_first_day = f.read()
cur.execute('rollback')
cur.execute(gcs_first_day)

time: 38.8 s


In [29]:
#labs
f = open('/Users/geickelb1/Documents/GitHub/mimic-code/concepts/firstday/labs-first-day.sql', 'r') 
labs_first_day = f.read()
cur.execute('rollback')
cur.execute(labs_first_day)

time: 3min 15s


In [30]:
#bloodgas arterial
f = open('/Users/geickelb1/Documents/GitHub/mimic-code/concepts/firstday/blood-gas-first-day-arterial.sql', 'r') 
blood_gas_arterial_first_day = f.read()
cur.execute('rollback')
cur.execute(blood_gas_arterial_first_day)

time: 1min 6s


In [21]:
#echo

f = open('/Users/geickelb1/Documents/GitHub/mimic-code/concepts/echo-data.sql', 'r') 
echodata = f.read()
#echodata = query_schema + echodata
#gcs_all48_sql = query_schema + gcsdata.format(tuple(patients))
#ventparam_all48_sql = ventparamdata#.format(tuple(icustay_id))  
cur.execute('rollback')
cur.execute(echodata)  #I THINK that if i just execute all the scripts that should be sufficient.
#ventparamTable= sql_exe_show('select * from ventparam_all48;')
#ventparam_all48_df= sql_to_df_icu(ventparamTable)

time: 30.7 s


In [31]:
#vent
f = open('/Users/geickelb1/Documents/GitHub/mimic-code/concepts/durations/ventilation-durations.sql', 'r') 
ventilation_durations_first_day = f.read()
cur.execute('rollback')
cur.execute(ventilation_durations_first_day)

time: 3min 2s


# attempting to run sofa

In [46]:
#sofa
f = open('/Users/geickelb1/Documents/GitHub/mimic-code/concepts/severityscores/sofa.sql', 'r') 
sofa = f.read()
cur.execute('rollback')
cur.execute(sofa)
sofaTable= sql_exe_show('select * from SOFA;')
sofa_df= sql_to_df_icu(sofaTable)

time: 40.9 s


In [41]:
sofa_df

,subject_id,hadm_id,icustay_id,sofa,respiration,coagulation,liver,cardiovascular,cns,renal
7,28448,177527,200012,1,NaN,1.0,NaN,0.0,0.0,0.0
12,21789,112486,200019,5,0.0,1.0,NaN,1.0,3.0,0.0
25,9960,159243,200036,8,NaN,1.0,NaN,1.0,4.0,2.0
41,28494,199133,200059,7,0.0,2.0,NaN,4.0,0.0,1.0
46,19891,112840,200065,15,4.0,3.0,1.0,4.0,0.0,3.0
60,61871,124231,200087,7,3.0,0.0,0.0,4.0,0.0,0.0
67,52619,175374,200099,7,2.0,2.0,NaN,1.0,1.0,1.0
68,11228,167826,200102,15,3.0,3.0,4.0,1.0,0.0,4.0
74,9993,150740,200109,3,0.0,0.0,0.0,3.0,0.0,0.0
75,20528,132580,200112,3,NaN,0.0,NaN,1.0,0.0,2.0


time: 30.8 ms


In [42]:
pd.DataFrame(sofa_df).to_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/02072018_sofa_firstday_df.csv')

time: 109 ms


In [55]:
#sofa_parameters
f = open('/Users/geickelb1/Documents/GitHub/mimic-code/concepts/severityscores/sofa_parameters.sql', 'r') 
sofa_parameters = f.read()
cur.execute('rollback')
cur.execute(sofa_parameters)
sofa_parametersTable= sql_exe_show('select * from SOFA_PARAMETERS;')
sofa_parameters_df= sql_to_df_icu(sofa_parametersTable)

time: 13.8 s


In [52]:
var_store= sofa_test_df

time: 645 µs


In [60]:
var_store2= sofa_test_df - var_store
var_store2.max()# this appears to show that adjusting the day window fom 0 to 1 day to -6hr to 48hr did not change any sofascore parameters... why? 

subject_id             0.0
hadm_id                0.0
icustay_id             0.0
meanbp_min             0.0
rate_norepinephrine    0.0
rate_epinephrine       0.0
rate_dopamine          0.0
rate_dobutamine        0.0
creatinine_max         0.0
bilirubin_max          0.0
platelet_min           0.0
pao2fio2_novent_min    0.0
pao2fio2_vent_min      0.0
urineoutput            0.0
mingcs                 0.0
dtype: float64

time: 13.3 ms


In [61]:
sofa_test_df

,subject_id,hadm_id,icustay_id,meanbp_min,rate_norepinephrine,rate_epinephrine,rate_dopamine,rate_dobutamine,creatinine_max,bilirubin_max,platelet_min,pao2fio2_novent_min,pao2fio2_vent_min,urineoutput,mingcs
3,31515,152958,200269,60.000000,NaN,NaN,5.0,NaN,1.4,NaN,147.0,NaN,NaN,360.0,15.0
5,24589,133207,200318,59.666698,0.050000,NaN,NaN,NaN,0.9,1.8,109.0,NaN,NaN,2075.0,15.0
7,20190,119413,200375,46.666698,NaN,NaN,2.5,NaN,0.4,0.1,711.0,NaN,220.000000,2730.0,14.0
12,24372,106081,200504,54.333302,NaN,NaN,7.5,NaN,1.9,NaN,256.0,NaN,NaN,835.0,8.0
16,28926,103306,200639,55.000000,0.105194,NaN,NaN,NaN,1.1,1.1,178.0,NaN,287.500000,750.0,15.0
20,11320,140887,200891,48.000000,0.300000,NaN,NaN,NaN,2.2,2.3,229.0,NaN,140.000000,1385.0,15.0
24,10076,198503,201006,1.000000,0.120000,NaN,NaN,NaN,0.8,NaN,306.0,NaN,91.666667,895.0,15.0
25,28571,107924,201121,66.000000,0.030000,NaN,NaN,NaN,0.9,0.7,205.0,NaN,NaN,2135.0,15.0
33,4096,124383,201361,44.333302,0.054477,NaN,NaN,NaN,2.8,0.2,340.0,NaN,NaN,491.0,14.0
41,32705,101962,201593,36.000000,1.000000,NaN,NaN,NaN,1.7,1.7,106.0,NaN,NaN,450.0,3.0


time: 41.8 ms


In [77]:
final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/final_pt_df2_5-04-18.csv', index_col=0)

patients= list(final_pt_df2['subject_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 61 ms


In [79]:
print(
    len(patients),
    len(icustay_id),
    len(final_pt_df2)
)

11493 11493 11493
time: 1.35 ms


In [75]:
pt_demographics = pd.read_csv('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/ptinfo_ICUsterile_pt_16_04-26-18.csv', index_col=0)


time: 326 ms


In [76]:
pt_demographics.loc[pt_demographics.loc[:,'icustay_id'].isin(icustay_id), :] #11490

,subject_id,hadm_id,hosp_admit,hosp_disch,admission_location,admission_type,first_admit_age,dob,gender,icustay_id,...,last_wardid,row_id,chartdate,culture_charttime,spec_itemid,spec_type_desc,sterile_dur_icu,culture_hadm_date_rng,first_sterile_date,culture_count
10,4.0,185777,2191-03-16 00:28:00,2191-03-23 18:41:00,EMERGENCY ROOM ADMIT,EMERGENCY,47.84,2143-05-12 00:00:00,F,294638,...,52,27,2191-03-18 00:00:00,2191-03-18 08:00:00,70012,BLOOD CULTURE,False,6 days 08:00:00.000000000,2191-03-16 00:00:00,13
23,9.0,150750,2149-11-09 13:06:00,2149-11-14 10:15:00,EMERGENCY ROOM ADMIT,EMERGENCY,41.79,2108-01-26 00:00:00,M,220597,...,15,47,2149-11-11 00:00:00,2149-11-11 12:00:00,70079,URINE,True,3 days 09:50:00.000000000,2149-11-10 09:40:00,7
35,19.0,109235,2108-08-05 16:25:00,2108-08-11 11:29:00,EMERGENCY ROOM ADMIT,EMERGENCY,300.00,1808-08-05 00:00:00,M,273430,...,23,69,2108-08-05 00:00:00,2108-08-05 20:42:00,70079,URINE,True,0 days 00:00:00.000000000,2108-08-05 20:42:00,1
42,21.0,111970,2135-01-30 20:50:00,2135-02-08 02:08:00,EMERGENCY ROOM ADMIT,EMERGENCY,87.44,2047-04-04 00:00:00,M,216859,...,52,79,2135-01-30 00:00:00,2135-01-30 19:05:00,70012,BLOOD CULTURE,False,4 days 09:08:00.000000000,2135-01-30 19:00:00,17
60,25.0,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,EMERGENCY ROOM ADMIT,EMERGENCY,58.95,2101-11-21 00:00:00,M,203487,...,7,100,2160-11-02 00:00:00,2160-11-02 08:51:00,70012,BLOOD CULTURE,True,0 days 00:05:00.000000000,2160-11-02 08:46:00,2
62,26.0,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,72.00,2054-05-04 00:00:00,M,244882,...,57,108,2126-05-10 00:00:00,2126-05-10 00:00:00,70079,URINE,False,0 days 05:50:00.000000000,2126-05-10 00:00:00,10
72,31.0,128652,2108-08-22 23:27:00,2108-08-30 15:00:00,TRANSFER FROM HOSP/EXTRAM,EMERGENCY,72.26,2036-05-17 00:00:00,M,254478,...,15,126,2108-08-23 00:00:00,2108-08-23 01:30:00,70012,BLOOD CULTURE,True,6 days 14:30:00.000000000,2108-08-23 00:00:00,17
89,33.0,176176,2116-12-23 22:30:00,2116-12-27 12:05:00,EMERGENCY ROOM ADMIT,EMERGENCY,82.39,2034-08-02 00:00:00,M,296681,...,12,140,2116-12-23 00:00:00,2116-12-23 19:00:00,70079,URINE,False,0 days 07:40:00.000000000,2116-12-23 19:00:00,6
276,62.0,116009,2113-02-15 00:19:00,2113-02-19 15:30:00,EMERGENCY ROOM ADMIT,EMERGENCY,68.77,2044-05-08 00:00:00,M,216609,...,7,438,2113-02-18 00:00:00,2113-02-18 11:17:00,70011,BLOOD CULTURE ( MYCO/F LYTIC BOTTLE),False,3 days 16:10:00.000000000,2113-02-15 00:00:00,8
284,64.0,172056,2143-03-03 09:25:00,2143-03-18 13:01:00,EMERGENCY ROOM ADMIT,EMERGENCY,26.68,2116-06-27 00:00:00,F,232593,...,52,442,2143-03-03 00:00:00,2143-03-03 11:48:00,70017,SEROLOGY/BLOOD,True,4 days 18:00:00.000000000,2143-03-03 00:00:00,14


time: 75.8 ms


In [71]:
len(pt_demographics.loc[pt_demographics.loc[:,'subject_id'].isin(patients), :]['subject_id'].unique()) #11493/11493 patients here,
#however 3 of the patients don't have their icustay_id in the icusterile 

11493

time: 9.88 ms
